In [1]:
 import numpy as np
from itertools import product
from scipy.spatial import cKDTree
import open3d as o3d

# -------------------------------
# STEP 1: Load and Subsample Data
# -------------------------------
pcd = o3d.io.read_point_cloud("/Users/judahlevin/UATX/Linear Algebra/Final Project/Models/bunny.ply")
X = np.asarray(pcd.points)
n_full = X.shape[0]

# Subsample to n_sample points
n_sample = 1000
indices = np.random.choice(n_full, n_sample, replace=False)
X = X[indices]
n = X.shape[0]
P = X - X.mean(axis=0)

# -------------------------------
# STEP 2: Generate Ground-Truth Transformed Cloud Q = O @ S @ P
# -------------------------------
def random_orthogonal_matrix(d=3):
    H = np.random.randn(d, d)
    Q, _ = np.linalg.qr(H)
    return Q

O = random_orthogonal_matrix()
perm = np.random.permutation(n)
S = np.eye(n)[perm]
Q_true = (O @ (S @ P).T).T
Q = Q_true - Q_true.mean(axis=0)

# -------------------------------
# STEP 3: Kolpakov's E–Init
# -------------------------------
E_P = P.T @ P
E_Q = Q.T @ Q
_, U_P = np.linalg.eigh(E_P)
_, U_Q = np.linalg.eigh(E_Q)
U0 = U_Q @ U_P.T

def reflection_group(d=3):
    return [np.diag(signs) for signs in product([-1, 1], repeat=d)]

best_U = None
best_error = float("inf")
best_indices = None
best_matched_P = None

for D in reflection_group():
    U_candidate = U0 @ U_P @ D @ U_P.T
    Q_init_candidate = Q @ U_candidate.T
    _, indices = cKDTree(P).query(Q_init_candidate)
    matched_P = P[indices]
    error = np.linalg.norm(matched_P - Q_init_candidate)
    if error < best_error:
        best_error = error
        best_U = U_candidate
        best_indices = indices
        best_matched_P = matched_P

# -------------------------------
# STEP 4: Final Aligned Output (No ICP)
# -------------------------------
Q_init = Q @ best_U.T

# -------------------------------
# STEP 5: Kolpakov Evaluation Metrics (E–Init only)
# -------------------------------
delta = np.linalg.norm(best_matched_P - Q_init)           # ✅ Alignment error
delta_o = np.linalg.norm(best_U - O)                      # ✅ Rotation error
true_sigma = np.argmax(S, axis=0)

# Construct ground-truth permutation matrix S
S_true = np.eye(n)[perm]  # shape (n, n)

# Construct estimated permutation matrix from best_indices
S_est = np.eye(n)[best_indices]  # shape (n, n)

# Compute Kolpakov's permutation error
delta_H = np.linalg.norm(S_est - S_true, ord='fro')**2 / (2 * n)

# -------------------------------
# Output Results
# -------------------------------
print("\n=== Kolpakov E–Init-Only Evaluation ===")
print(f"δ        (alignment error):      {delta:.6f}")
print(f"δ_o      (rotation error):       {delta_o:.6f}")
print(f"δ_H      (permutation error):    {delta_H:.6f}")


=== Kolpakov E–Init-Only Evaluation ===
δ        (alignment error):      0.470099
δ_o      (rotation error):       2.000000
δ_H      (permutation error):    0.999000


In [2]:
# VISUALIZATION
import open3d as o3d
import numpy as np

# Assume Q and Q_init are already defined

# Convert to Open3D point clouds
pcd_Q = o3d.geometry.PointCloud()
pcd_Q_init = o3d.geometry.PointCloud()

pcd_Q.points = o3d.utility.Vector3dVector(Q)
pcd_Q_init.points = o3d.utility.Vector3dVector(Q_init)

# Color coding
# Warm coral for Q
pcd_Q.paint_uniform_color([1.0, 0.5, 0.5])  # Soft coral pink

# Cool turquoise for Q_init
pcd_Q_init.paint_uniform_color([0.4, 0.8, 0.8])  # Light teal

# Display
o3d.visualization.draw_geometries(
    [pcd_Q, pcd_Q_init],
    window_name="Q (pink) vs Q_init (teal)",
    width=800,
    height=600,
    point_show_normal=False
)
